# First Pose Estimation Project
 Creating an algorithm using open source [OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose)

## Importing needed packages 
- OpenCV as ```cv2```
- Numpy as ```np```
- Pandas as ```pd```


In [3]:
import sys
import cv2
import os
from sys import platform
import argparse
import numpy as np
#import pytesseract

### Neural Network trained on the Coco dataset:
**Given keypoints from Coco dataset**

| Keypoints      | Numbering using by Coco Dataset |
| -----------    | -----------                     |
| Nose           | 0                               |
| Neck           | 1                               |
| Right Shoulder | 2                               |
| Right Elbow    | 3                               |
| Right Wrist    | 4                               |
| Left Shoulder  | 5                               |
| Left Elbow     | 6                               |
| Left Wrist     | 7                               |
| Right Hip      | 8                               |
| Right Knee     | 9                               |
| Right Ankle    | 10                              |
| Left Hip       | 11                              |
| Left Knee      | 12                              |
| Left Ankle     | 13                              |
| Right Eye      | 14                              |
| Left Eye       | 15                              |
| Right Ear      | 16                              |
| Left Ear       | 17                              |
| Background     | 18                              |

In [4]:
BODY_PARTS = {  "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4, "LShoulder": 5, "LElbow": 6, 
                "LWrist": 7, "RHip": 8, "RKnee": 9, "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, 
                "REye": 14, "LEye": 15, "REar": 16, "LEar": 17, "Background": 18}

POSE_PAIRS = [  ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"], ["RElbow", "RWrist"],
                ["LShoulder", "LElbow"], ["LElbow", "LWrist"], ["Neck", "RHip"], ["RHip, RKnee"], ["RKnee", "RAnkle"],
                ["Neck", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
                ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]]

In [5]:
protoFile = "/Users/Maajid/openpose/input/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "/Users/Maajid/openpose/input/pose_iter_160000.caffemodel"
# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
threshold = 0.2

## Run the following cell to generate pose estimation on an image
- for a video please run the cell following the next


In [6]:
# TODO make this a function so that image size and width can be modulated
# Read image
frame = cv2.imread("PoseVideos/img1.JPG", cv2.IMREAD_UNCHANGED)

# Specify the input image dimensions
inWidth = 400
inHeight = 400

#imput frame len + width
photo_height = frame.shape[0]
photo_width = frame.shape[1]

#cv2.imshow("frame", frame)
# Prepare the frame to be fed to the network
inpBlob = cv2.dnn.blobFromImage(
	frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

# Set the prepared object as the input blob of the network
net.setInput(inpBlob)

output = net.forward()
#see what the size means is for 1,44,50,50 --> what is it outputting
print(len(BODY_PARTS))
print(output.shape[1])


19
44


In [16]:
#assert(len(BODY_PARTS) == output.shape[1])
points = [(None, None)]

for i in range(len(BODY_PARTS)):
    #slice heatmap of correspoinding body's part
    heatMap = output[0, i, :, :]
    _, conf, _, points = cv2.minMaxLoc(heatMap)
    x = (photo_width * points[0] / output.shape[3])
    y = (photo_height * points[1] / output.shape[2])
    list(points)
    points.append((int(x), int(y)) if conf > threshold else None)

for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert(partFrom in BODY_PARTS)
    assert(partTo in BODY_PARTS)

    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]

    if points[idFrom] and points[idTo]:
        cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv2.ellipse(frame, points[idFrom], (3,3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
        cv2.ellipse(frame, points[idTo], (3,3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

t, _ = net.getPerfProfile()
cv2.imshow("Output", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


AttributeError: 'tuple' object has no attribute 'append'

### Running algorithm on Video file
- implements same logic as an image input, but goes through a while-loop

In [ ]:
cap = cv2.VideoCapture(0)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
count = 0
index = 0
whole_body_coordinates = {}
for x in range(15):
    whole_body_coordinates[x] = []
while (cap.isOpened()):
    ret, frame = cap.read()
    

## Generating an output
- Draws over the image and allows for a visualization of the keypoints and the lines that connect the keypoints together

In [8]:

H = output.putshape[2]
W = output.shape[3]
# Empty list to store the detected keypoints
points = []
for i in range(len()):
	# confidence map of corresponding body's part.
	probMap = output[0, i, :, :]

	# Find global maxima of the probMap.
	minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

	# Scale the point to fit on the original image
	
	# ToDo: whatever characteristic of my image

	x = (inWidth * point[0]) / W
	y = (inHeight * point[1]) / H

	if prob > 0.5: #how confident i want the predictions to be -- a
		cv2.circle(frame, (int(x), int(y)), 15, (0, 255, 255),
				thickness=-1, lineType=cv.FILLED)
		cv2.putText(frame, "{}".format(i), (int(x), int(
			y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)

		# Add the point to the list if the probability is greater than the threshold
		points.append((int(x), int(y)))
	else:
		points.append(None)

cv2.imshow("Output-Keypoints", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

for pair in POSE_PAIRS:
	partA = pair[0]
	partB = pair[1]

	if points[partA] and points[partB]:
		cv2.line(frame, points[partA], points[partB], (0, 255, 0), 3)


'\nH = output.putshape[2]\nW = output.shape[3]\n# Empty list to store the detected keypoints\npoints = []\nfor i in range(len()):\n\t# confidence map of corresponding body\'s part.\n\tprobMap = output[0, i, :, :]\n\n\t# Find global maxima of the probMap.\n\tminVal, prob, minLoc, point = cv2.minMaxLoc(probMap)\n\n\t# Scale the point to fit on the original image\n\t\n\tToDo: whatever characteristic of my image\n\n\tx = (inWidth * point[0]) / W\n\ty = (inHeight * point[1]) / H\n\n\tif prob > 0.5: #how confident i want the predictions to be -- a\n\t\tcv2.circle(frame, (int(x), int(y)), 15, (0, 255, 255),\n\t\t\t\tthickness=-1, lineType=cv.FILLED)\n\t\tcv2.putText(frame, "{}".format(i), (int(x), int(\n\t\t\ty)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)\n\n\t\t# Add the point to the list if the probability is greater than the threshold\n\t\tpoints.append((int(x), int(y)))\n\telse:\n\t\tpoints.append(None)\n\n#cv2.imshow("Output-Keypoints", frame)\n#cv2.waitKey(0)\

In [4]:
onesDigit = 29 % 10
tensDigit = (29 - onesDigit) / 10
print(onesDigit + tensDigit)

11.0
